In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Training data
df1=pd.read_csv("wind_power_flow_data1.csv")
df2=pd.read_csv("power_flow_conv1.csv")

#print(df2)
#print(df2)

col1 = df1['MWh']
col21 = df2[['Average','Variability']]



#Validation data
df3=pd.read_csv("wind_power_flow_data2.csv")
df4=pd.read_csv("power_flow_conv2.csv")

col3= df3['MWh']
col41 = df4[['Average','Variability']]




#Training data
wind_power1 = col1.as_matrix()
classify_output1 = col21.as_matrix()

#converting to float32
wind_power1.astype(float, copy=False)
classify_output1.astype(float, copy=False)

#Validation data
wind_power2 = col3.as_matrix()
classify_output2 = col41.as_matrix()

#converting to float32
wind_power2.astype(float, copy=False)
classify_output2.astype(float, copy=False);


#print("Number of values:",np.count_nonzero(average_power1))
print("Number of values in training output:",classify_output1.shape)
print("Number of values in validation output:",classify_output2.shape)

Number of values in training output: (350, 2)
Number of values in validation output: (350, 2)


In [3]:
#pmax = np.amax(wind_power)
wind_power_max = 10000    #For normalization
batch_size = 350           #Number of training samples
output_size =2
wind_power_reshape = np.resize(wind_power1/wind_power_max,(batch_size,1,24,1))
classify_output1_reshape = np.resize(classify_output1,(batch_size,output_size))
#Print reshaped input and output training data
print(wind_power_reshape)
print(classify_output1_reshape)
#Validation data
validate_batch = 350    #Number of validation samples
wind_power_reshape_validate = np.resize(wind_power2/wind_power_max,(validate_batch,1,24,1))
classify_output2_reshape_validate = np.resize(classify_output2,(validate_batch,output_size))


[[[[ 0.391694]
   [ 0.388492]
   [ 0.386615]
   ..., 
   [ 0.283319]
   [ 0.240907]
   [ 0.22122 ]]]


 [[[ 0.226892]
   [ 0.237066]
   [ 0.261512]
   ..., 
   [ 0.329363]
   [ 0.34579 ]
   [ 0.383075]]]


 [[[ 0.411239]
   [ 0.459406]
   [ 0.476016]
   ..., 
   [ 0.948027]
   [ 0.943393]
   [ 0.963431]]]


 ..., 
 [[[ 0.655609]
   [ 0.634282]
   [ 0.615162]
   ..., 
   [ 0.370805]
   [ 0.380299]
   [ 0.383783]]]


 [[[ 0.378272]
   [ 0.393201]
   [ 0.391862]
   ..., 
   [ 0.622414]
   [ 0.643337]
   [ 0.666998]]]


 [[[ 0.684243]
   [ 0.690735]
   [ 0.69413 ]
   ..., 
   [ 0.43058 ]
   [ 0.412425]
   [ 0.395917]]]]
[[ 0.32123958  0.14991743]
 [ 0.27448967  0.1749647 ]
 [ 0.77340729  0.53143856]
 [ 0.87074179  0.22627937]
 [ 0.81383646  0.38703368]
 [ 0.71497221  0.29172574]
 [ 0.20066887  0.28838797]
 [ 0.13548058  0.25268615]
 [ 0.51452454  0.27782923]
 [ 0.33550529  0.38662274]
 [ 0.52240825  0.26215155]
 [ 0.70465379  0.53572525]
 [ 0.60499817  0.53420977]
 [ 0.80302458  0.13978399

In [4]:
#Power curve parameteres
N =24 #Window
Padding =0
Stride =1
Fn1 = 2 #Number of features
Fn2 = 2 #Number of features
Filter_size = 6
Filter1_out = int((N -Filter_size + 2*Padding)/Stride  +1)
Filter2_out = int((Filter1_out -Filter_size + 2*Padding)/Stride +1)

O1 = 2 # Neurons in fully connected layer
O2 = 2 # Number of outputs
print("Number of outputs after 1st convolution:",Filter1_out)
print("Number of outputs after 2nd convolution:",Filter2_out)
training_iterations=100000
display_iterations = 10000
learning_rate1 = 1e-3

Number of outputs after 1st convolution: 19
Number of outputs after 2nd convolution: 14


In [5]:
#Input
power = tf.placeholder(tf.float32,shape=(batch_size,1,24,1))#4x4 image (4D tensor = [1,1,6,1] = [batch size, width, height, number of channels]
#Output
y = tf.placeholder(tf.float32)

#Convolution layer weights
power_filter1 = tf.Variable(tf.random_normal([1,Filter_size,1,Fn1])) #2x2 filter (4D tensor = [3,3,1,1] = [width, height, channels, number of filters]) 
power_filter2 = tf.Variable(tf.random_normal([1,Filter_size,Fn1,Fn2])) #2x2 filter (4D tensor = [3,3,1,1] = [width, height, channels, number of filters]) 
#Convolution layer biases
b_conv1 = tf.Variable(tf.constant(0.1,shape=[Fn1]))
b_conv2 = tf.Variable(tf.constant(0.1,shape=[Fn2]))
#Fully connected layer weights
W_fc1 = tf.Variable(tf.random_normal([1*Filter2_out*Fn2,O1])) # Fully connected layer (2D tensor = [size of featur map*number of feature map,Number of neurons]) 
b_fc1 = tf.Variable(tf.constant(0.1,shape=[O1]))
#Output layer weights
W_fc2 = tf.Variable(tf.random_normal([O1,O2])) # Fully connected layer (2D tensor = [Number of input neurons,Number of output neurons]) 
b_fc2 = tf.Variable(tf.constant(0.1,shape=[O2]))

In [6]:
#Convolution layer
convolve1 = tf.nn.conv2d(power, power_filter1, strides=[1, 1, 1, 1], padding='VALID')
h_conv1 = tf.nn.relu(convolve1+b_conv1)
convolve2 = tf.nn.conv2d(h_conv1, power_filter2, strides=[1, 1, 1, 1], padding='VALID')
h_conv2 = tf.nn.relu(convolve2+b_conv2)
#Fully connected
layer2_matrix = tf.reshape(h_conv2, [-1, 1*Filter2_out*Fn2])
matmul_fc1=tf.matmul(layer2_matrix, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)
#Output layer
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
output_layer = matmul_fc2  #Applying linear activation


In [7]:
#Decreasing learning rate
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = learning_rate1
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,10000, 0.96, staircase=True)
# Passing global_step to minimize() will increment it at each step.

#Loss function
mean_square = tf.reduce_sum(tf.square(y-output_layer))
#train_step = tf.train.AdagradOptimizer(learning_rate).minimize(mean_square,global_step=global_step)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square,global_step=global_step)

#Operation to save variables
saver = tf.train.Saver()


In [8]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    power1 = np.reshape(wind_power_reshape[0:batch_size],(-1,1,24,1))
    y1 = classify_output1_reshape[0:batch_size]
    
    print("loss at start:",sess.run([mean_square],feed_dict={power:power1,y:y1}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={power:power1,y:y1})
        if i%display_iterations ==0:
            print("loss is:",sess.run([mean_square],feed_dict={power:power1,y:y1}),"at itertion:",i)
            print("learning rate is:",sess.run([learning_rate]),"at itertion:",i)   
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("loss:",sess.run([mean_square],feed_dict={power:power1,y:y1}))
    

loss at start: [139.45892]
loss is: [137.0815] at itertion: 0
learning rate is: [0.001] at itertion: 0
loss is: [6.3683462] at itertion: 10000
learning rate is: [0.00096000003] at itertion: 10000
loss is: [6.3536558] at itertion: 20000
learning rate is: [0.00092160003] at itertion: 20000
loss is: [6.3522353] at itertion: 30000
learning rate is: [0.00088473596] at itertion: 30000
loss is: [6.351234] at itertion: 40000
learning rate is: [0.00084934651] at itertion: 40000
loss is: [6.3521814] at itertion: 50000
learning rate is: [0.00081537262] at itertion: 50000
loss is: [6.351593] at itertion: 60000
learning rate is: [0.00078275776] at itertion: 60000
loss is: [6.3506613] at itertion: 70000
learning rate is: [0.0007514474] at itertion: 70000
loss is: [6.3505363] at itertion: 80000
learning rate is: [0.00072138949] at itertion: 80000
loss is: [6.3502507] at itertion: 90000
learning rate is: [0.00069253391] at itertion: 90000
Model saved in file: /tmp/model.ckpt
loss: [6.3502479]


In [9]:
#Retrieve and continue session
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    power1 = np.reshape(wind_power_reshape[0:batch_size],(-1,1,24,1))
    y1 = classify_output1_reshape[0:batch_size]
    
    
    print("loss:",sess.run([mean_square],feed_dict={power:power1,y:y1}))
    
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={power:power1,y:y1})
        if i%display_iterations ==0:
             print("loss is:",sess.run([mean_square],feed_dict={power:power1,y:y1}),"at itertion:",i)
        
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("loss:",sess.run([mean_square],feed_dict={power:power1,y:y1}))
    yNN_trained =sess.run([output_layer],feed_dict={power:power1})

Model restored.
loss: [6.3502479]
loss is: [6.3500605] at itertion: 0
loss is: [6.3501744] at itertion: 10000
loss is: [6.3499079] at itertion: 20000
loss is: [6.3496828] at itertion: 30000
loss is: [6.3496141] at itertion: 40000
loss is: [6.3493924] at itertion: 50000
loss is: [6.349216] at itertion: 60000
loss is: [6.3491611] at itertion: 70000
loss is: [6.3487873] at itertion: 80000
loss is: [6.3485479] at itertion: 90000
Model saved in file: /tmp/model.ckpt
loss: [6.3484631]


In [10]:
with tf.Session() as sess:
    # Restore variables from disk for validation.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    power2 = np.reshape(wind_power_reshape_validate[0:validate_batch],(-1,1,24,1))
    y2 = classify_output2_reshape_validate[0:validate_batch]
    print("loss:",sess.run([mean_square],feed_dict={power:power2,y:y2}))
    print("output:",sess.run([output_layer],feed_dict={power:power2}))
    yNN_validate = sess.run([output_layer],feed_dict={power:power2})
    

Model restored.
loss: [5.9791541]
output: [array([[ 0.22763428,  0.27594358],
       [ 0.33135596,  0.29770559],
       [ 0.55898553,  0.34546492],
       [ 0.64678097,  0.92180431],
       [ 0.6472708 ,  0.36398822],
       [ 0.50295687,  0.47633994],
       [ 0.61811101,  0.70284843],
       [ 0.66952097,  0.36865655],
       [ 0.65705091,  0.6287927 ],
       [ 0.73503751,  0.38240266],
       [ 0.61153275,  0.35648996],
       [ 0.66237962,  0.64689004],
       [ 0.35999697,  0.30371481],
       [ 0.18018019,  0.26598713],
       [ 0.56266916,  0.34623778],
       [ 0.67510432,  0.43280298],
       [ 0.19836888,  0.28856856],
       [ 0.73355186,  0.38209099],
       [ 0.83121228,  0.40258127],
       [ 0.51452488,  0.33613658],
       [ 0.66130465,  0.39478433],
       [ 0.14617798,  0.31659478],
       [ 0.56352925,  0.34641826],
       [ 0.46324807,  0.32537809],
       [ 0.55481189,  0.34458923],
       [ 0.4253048 ,  0.31741714],
       [ 0.45046228,  0.81681901],
       [ 0.1

In [11]:
#Converting to numpy arrays
yNN_validate=np.asarray(yNN_validate)
yNN_trained=np.asarray(yNN_trained)


In [12]:
#Saving CNN output
np.savetxt("/tmp/NN_validate_output1.csv", yNN_validate[:,:,0], delimiter=",")
np.savetxt("/tmp/NN_validate_output2.csv", yNN_validate[:,:,1], delimiter=",")

np.savetxt("/tmp/NN_trained_output1.csv", yNN_trained[:,:,0], delimiter=",")
np.savetxt("/tmp/NN_trained_output2.csv", yNN_trained[:,:,1], delimiter=",")



In [13]:
with tf.Session() as sess:
    # Restore variables from disk 
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    print("Conv filter1:",sess.run([power_filter1]))
    print("Conv filter2:",sess.run([power_filter2]))
    print("Conv bias 1:",sess.run([b_conv1]))
    print("Conv bias 2:",sess.run([b_conv1]))
    print("Full connected layer 1 weights:",sess.run([W_fc1]))
    print("Full connected layer 1 bias:",sess.run([b_fc1]))
    print("Full connected layer 2 weights:",sess.run([W_fc2]))
    print("Full connected layer 2 bias:",sess.run([b_fc2]))

Model restored.
Conv filter1: [array([[[[-0.26336747, -0.47294194]],

        [[-0.51627028, -0.59489781]],

        [[ 0.41553342, -2.24790382]],

        [[-1.03019333, -1.95349717]],

        [[ 0.37584552,  0.05892721]],

        [[-0.22681765,  0.75300574]]]], dtype=float32)]
Conv filter2: [array([[[[-0.67893624, -0.09012523],
         [-1.50055158,  1.26097167]],

        [[ 0.14149183, -0.30843493],
         [ 1.33816707, -0.99697292]],

        [[ 1.37863398,  1.60072029],
         [-1.57425201,  1.50976384]],

        [[-1.11969805, -0.54292911],
         [ 1.2430557 ,  0.06524333]],

        [[ 0.06871944,  0.65121651],
         [-2.53878093,  1.06782794]],

        [[-0.19633895,  0.39411393],
         [ 0.49591494,  0.53815144]]]], dtype=float32)]
Conv bias 1: [array([ 1.13493621,  0.03495971], dtype=float32)]
Conv bias 2: [array([ 1.13493621,  0.03495971], dtype=float32)]
Full connected layer 1 weights: [array([[ 0.95681661,  0.25760433],
       [-0.29264909,  0.00298006],